# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [2]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [3]:
# YOUR CODE HERE
cities = cities.rename(columns = {'Population (2016 est.)[8]':'Population'})
l = ['NFL','MLB','NBA','NHL']
for column in l:
    cities[column] = cities[column].str.replace("\[.*\]","")
    cities[column][(cities[column] == '—') | (cities[column] == '')] = np.nan

In [29]:
cities

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers
7,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche


In [4]:
def transform_cities(col_of_interest):
    
    pattern = '([A-Z0-9][a-z0-9]*\s\w*|[A-Z]{0,1}[a-z0-9]*)([A-Z0-9][a-z0-9]*\s\w*|[A-Z]{0,1}[a-z]*)(\w*)'
    df = cities[col_of_interest].str.extract(pattern)
    df['Metropolitan area'] = cities['Metropolitan area']
    df = pd.melt(df, id_vars=['Metropolitan area'], value_name = 'Teams').drop(columns = 'variable')
    df = df.dropna()
    df = df[df['Teams'] != '']
    df=pd.merge(df,cities,how='left',on = ['Metropolitan area']).iloc[:,0:3]
    return df

In [18]:
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL
6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL
7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL
8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL
9,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,2018,NHL


In [5]:
# YOUR CODE HERE

def transform_nhl():
    
    nhl_df['team'] = nhl_df['team'].str.replace('\*','')
    df = nhl_df[nhl_df['year'] == 2018][['team','L','W']].rename(columns = {'team':'Teams'})
    df = df[df['Teams'].str.contains('Division')==False] # Another way: #df = df.drop(df[df['Teams'].str.match('.*Division',na=False)].index)
    df['Teams'] = df['Teams'].str.replace('[\w.]*\s','')
    
    d = {'Wings':"Red Wings","Leafs":"Maple Leafs","Jackets":"Blue Jackets","Knights":"Golden Knights"} 
    for old_name, new_name in d.items():
        df['Teams'][df['Teams'] == old_name] = new_name

    df = df.astype({'L': int, "W": int})
    df['W/L Ratio'] = df['W'] / (df['L'] + df['W'])    
    df = df.astype({'W/L Ratio': float})
    return df

def nhl_correlation(need_df = False): 
    
    df_cities = transform_cities('NHL')
    df_nhl = transform_nhl()
    
    df_merged = pd.merge(df_cities,df_nhl, how = 'inner', on = 'Teams')
    df_merged = df_merged.astype({'Population': int})
    df_merged = df_merged.groupby('Metropolitan area').agg({'W/L Ratio': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = df_merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df_merged['W/L Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    # the below if statement is necessary for question 5 (I included the variable need_df, which was not initially
    # in the assignment, to avoid writing uncessary code for question 5.)
    if need_df == True:
        df_merged = df_merged.drop('Population',axis = 1).rename(columns = {"W/L Ratio":"NHL"})
        return df_merged
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [6]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [23]:
nba_df=pd.read_csv("assets/nba.csv")
nba_df

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA
5,Miami Heat* (6),44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA
6,Milwaukee Bucks* (7),44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA
7,Washington Wizards* (8),43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA
8,Detroit Pistons (9),39,43,0.47600000000000003,20.0,103.8,103.9,-0.26,2018,NBA
9,Charlotte Hornets (10),36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA


In [7]:
# YOUR CODE HERE

def transform_nba():
    
    nba_df['team'] = nba_df['team'].str.replace('\*{0,1}\s\(.*\)','')
    df = nba_df[nba_df['year'] == 2018][['team','W/L%']].rename(columns = {'team':'Teams'})
    df['Teams'] = df['Teams'].str.replace('[\w.]*\s','')
    
    df['Teams'][df['Teams'] == 'Blazers'] = 'Trail Blazers'
    df = df.astype({'W/L%': float})
    return df


def nba_correlation(need_df = False):
    
    df_cities = transform_cities('NBA')
    df_nba = transform_nba()
    
    df_merged = pd.merge(df_cities,df_nba, how = 'outer', on = 'Teams')
    df_merged = df_merged.astype({'Population': int})
    df_merged = df_merged.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    
    population_by_region = df_merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df_merged['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    # the below if statement is necessary for question 5 (I included the variable need_df, which was not initially
    # in the assignment, to avoid writing uncessary code for question 5)
    if need_df == True:
        df_merged = df_merged.drop('Population',axis = 1).rename(columns = {"W/L%":"NBA"})
        return df_merged
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17636350642182938

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [28]:
mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB
5,Cleveland Indians,91,71,0.562,--,2018,MLB
6,Minnesota Twins,78,84,0.481,13.0,2018,MLB
7,Detroit Tigers,64,98,0.395,27.0,2018,MLB
8,Chicago White Sox,62,100,0.383,29.0,2018,MLB
9,Kansas City Royals,58,104,0.358,33.0,2018,MLB


In [8]:
# YOUR CODE HERE

def transform_mlb():
        
    df = mlb_df[mlb_df['year'] == 2018][['team','W-L%']].rename(columns = {'team':'Teams'})
    df['Teams'] = df['Teams'].str.replace('[\w.]*\s','')
    
    df.loc[0,'Teams'] = 'Red Sox'
    df.loc[8,'Teams'] = 'White Sox'
    df.loc[3,'Teams'] = 'Blue Jays'
    df = df.astype({'W-L%': float})
    return df

def mlb_correlation(need_df = False): 
    # YOUR CODE HERE
    df_cities = transform_cities('MLB')
    df_mlb = transform_mlb()

    df_merged = pd.merge(df_cities,df_mlb, how = 'inner', on = 'Teams')
    df_merged = df_merged.astype({'Population': int})
    df_merged = df_merged.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = df_merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df_merged['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    # the below if statement is necessary for question 5 (I included the variable need_df, which was not initially
    # in the assignment, to avoid writing uncessary code for question 5.)
    if need_df == True:
        df_merged = df_merged.drop('Population',axis = 1).rename(columns = {"W-L%":"MLB"})
        return df_merged
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [30]:
nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018
5,AFC North,AFC North,NFL,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,2018
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018
7,1.7,6,NFL,4.3,3.9,360,68,428,5.6,1.3,1,9,.594,Pittsburgh Steelers,2018
8,0.6,8,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,.469,Cleveland Browns,2018
9,-3.4,10,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,.375,Cincinnati Bengals,2018


In [9]:
# YOUR CODE HERE

def transform_nfl():
    nfl_df['team'] = nfl_df['team'].str.replace('[\*\+]','')
    df = nfl_df[nfl_df['year'] == 2018][['team','W-L%']].rename(columns = {'team':'Teams'})
    df = df.drop(df[df['Teams'].str.match('AFC|NFC',na=False)].index)
    df['Teams'] = df['Teams'].str.replace('[\w.]*\s','')
    df = df.astype({'W-L%': float})
    return df

def nfl_correlation(need_df = False): 
    
    df_cities = transform_cities('NFL')
    df_nfl = transform_nfl()
    
    df_merged = pd.merge(df_cities,df_nfl, how = 'inner', on = 'Teams')
    df_merged = df_merged.astype({'Population': int})
    df_merged = df_merged.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = df_merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df_merged['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    # the below if statement is necessary for question 5 (I included the variable need_df, which was not initially
    # in the assignment, to avoid writing uncessary code for question 5.)
    if need_df == True:
        df_merged = df_merged.drop('Population',axis = 1).rename(columns = {"W-L%":"NFL"})
        return df_merged
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [10]:
nfl_correlation()

0.004282141436393017

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [11]:
df_nfl = nfl_correlation(True)
df_nba = nba_correlation(True)
df_nhl = nhl_correlation(True)
df_mlb = mlb_correlation(True)

def sports_team_performance():
    # YOUR CODE HERE
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    
    d = {
        'NFL': df_nfl,
        'NBA': df_nba,
        'NHL': df_nhl,
        'MLB': df_mlb }
    
    for team_A in d.keys():
        for team_B in d.keys():
            if team_A != team_B :
                df_merged = pd.merge(d[team_A],d[team_B], how = 'inner', on = 'Metropolitan area')
                p_values.loc[team_A, team_B]=stats.ttest_rel(df_merged[team_A], df_merged[team_B])[1]

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


In [12]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
